# Init

In [ ]:
import uproot
import awkward as ak
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd

import hist
from hist import Hist
import mplhep

from pathlib import Path

In [ ]:
paths = ['outputs/final/out_1M_beauty_CA_R04_thermalBckg_true.root', 
         'outputs/final/out_1M_lf_CA_R04_thermalBckg_true.root']

In [ ]:
# paths = ['outputs/out_b_100k_true.root', 'outputs/out_lf_100k_true.root']

In [ ]:
flavours_cuts = {'beauty':'(pycode[:,0] >= 123) & (pycode[:,0] <= 124) & (hasBeauty[:,0] == 1)',
                'udsg':'(pycode[:,0] >= 111) & (pycode[:,0] <= 116) & (hasCharm[:,0] == 0) & (hasBeauty[:,0] == 0) & ((abs(ppid[:,0]) < 4) | (abs(ppid[:,0]) == 21))',
                'uds':'(pycode[:,0] >= 111) & (pycode[:,0] <= 116) & (hasCharm[:,0] == 0) & (hasBeauty[:,0] == 0) & (abs(ppid[:,0]) < 4)',
                'g':'(pycode[:,0] >= 111) & (pycode[:,0] <= 116) & (hasCharm[:,0] == 0) & (hasBeauty[:,0] == 0) & (abs(ppid[:,0]) == 21)',
                }


LABELS = {'lnOneOverTheta' : r'$\ln 1/\theta$',
          'lnkt': r'$\ln \it{k}_{\rm T}$',
          'Q_b_udsg':r'$Q\,(b,\;udsg)$',
          'Q_comb_py': r'$Q\,(emb,\;PYTHIA)$',
          'ptjet' : r'$p_{\rm\,T}^{\rm \,jet}$',
          'erad' : r'$E_{\rm\,rad}$',
          'gevc': r'GeV$\,/\,\it{c}$',
          'gev': r'GeV',
         }


colors = {'default':'k','beauty':'red', 'udsg':'navy', }
def darker(c, frac=0.3):
    import matplotlib.colors
    return [min(max(x-frac, 0),1) for x in matplotlib.colors.to_rgb(c)]

def brighter(c, frac=0.3):
    return darker(c, frac=-frac)

stylings = {'beauty_py': {'color':matplotlib.colors.to_hex(brighter('red')), 'ls':'-', 'marker': 'D'},
           'beauty_comb': {'color':matplotlib.colors.to_hex(darker('red')), 'ls':':', 'lw':2, 'marker': 'D', 'fillstyle':'none'},
           'udsg_py': {'color':matplotlib.colors.to_hex(brighter('navy')), 'ls':'-', 'marker': 's'},
           'udsg_comb': {'color':matplotlib.colors.to_hex(darker('navy')), 'ls':':', 'lw':2, 'marker': 's', 'fillstyle':'none'},
            
           'ratio_py': {'color':'gray', 'ls':'-', 'marker': 'o', 'fillstyle':'full'},
           'ratio_comb': {'color': 'k', 'ls':':', 'lw':2,  'marker': 'o', 'fillstyle':'none'},
           'ratio_beauty': {'color':matplotlib.colors.to_hex(('r')), 'ls':'-', 'marker': 'D', 'fillstyle':'full'},
           'ratio_udsg': {'color':matplotlib.colors.to_hex(('navy')), 'ls':'-', 'marker': 's', 'fillstyle':'full'},
           
           }

In [ ]:
def to_dtype(arr,dtype=int):
    # no fast and easy way to convert from floats to int
    return ak.values_astype(arr, int)

def groomer2str(g):
    return g.replace('soft_drop', 'SD').replace('dynamical', 'DyG')

def var2str(v):
    if v == 'lund_Delta':
        return '\\theta'
    if v == 'lund_z':
        return 'z'
    if v == 'lund_kt':
        return '#it{k}_{T}'
    if v == 'log(lund_kt)':
        return 'log(#it{k}_{T})'
    if v == 'lund_p_pt':
        return '#it{E}_{rad}'
    return v

def print_arr_summary(arr, qs=(0.01,0.99)):
    print(f'mean={np.mean(arr):.3f}, std={np.std(arr):.3f}, q{"-".join(str(int(q*100)) for q in qs)}={" - ".join(str(np.round(np.quantile(arr,q),2)) for q in qs)}\n{arr}')

In [ ]:
class Groomer:
    def __init__(self, cut_groom=None, cut_pre=None, cut_post=None):
        self.descr = f'precut = {cut_pre} | groom = {cut_groom} | postcut = {cut_post}'
        self.descr_short = (f'precut = {cut_pre} | ' if cut_pre else '')+f' groom = {cut_groom} ' + (f' | postcut = {cut_post}' if cut_post else '')
        self.cut_pre = cut_pre if cut_pre is not None else 'z > -999'
        self.cut_post= cut_post  if cut_post is not None else 'z > -999'
        self.cut_groom = cut_groom
        self.iterable_variables = ['z', 'Delta', 'p_pt', 'kt', 'kappa', 'Index', 'numArr']
    
    def __str__(self):
        return self.descr
    def __repr__(self):
        return self.__str__()
    def get_label(self):
        return self.descr
    def get_label_short(self):
        return self.descr_short
    
    
    def _convert_expression(self, exp, evt_lvl, idx_str=''):
        for q in self.iterable_variables:
            exp = exp.replace(q, f"arrs[\'lund_{evt_lvl}{q}\']{idx_str}")
        return exp
    
    def groom(self, arrs, evt_lvl):
        if self.cut_groom is None:
            return ValueError
        

        
        if f'lund_{evt_lvl}Index' not in arrs.fields:
            arrs[f'lund_{evt_lvl}Index'] = ak.local_index(arrs[f'lund_{evt_lvl}z'])
        if f'lund_{evt_lvl}num' not in arrs.fields:
            arrs[f'lund_{evt_lvl}num'] = ak.num(arrs[f'lund_{evt_lvl}z'])
        if f'lund_{evt_lvl}numArr' not in arrs.fields:
            n = ak.num(arrs[f'lund_{evt_lvl}z']) 
            arrs[f'lund_{evt_lvl}numArr'] = n * arrs[f'lund_{evt_lvl}Index']**0 
            
        from copy import deepcopy
        arrs = deepcopy(arrs)

        mask_pre = eval(self._convert_expression(self.cut_pre, evt_lvl))   
        grooming_quantity = eval(self._convert_expression(self.cut_groom, evt_lvl))
        groomed_split_idx = ak.argmax(ak.mask(grooming_quantity, mask_pre), 
                              axis=1, keepdims=1)
        mask_post = eval(self._convert_expression(self.cut_post, evt_lvl, '[groomed_split_idx]'))
        
        idx = ak.flatten(ak.mask(groomed_split_idx, mask_post))
#         fields = [f'lund_{evt_lvl}Delta', f'lund_{evt_lvl}z', f'lund_{evt_lvl}kt', f'lund_{evt_lvl}p_pt', f'lund_{evt_lvl}kappa'
        fields = [f'lund_{evt_lvl}{var}' for var in self.iterable_variables if f'lund_{evt_lvl}{var}' in arrs.fields]
        vals = ak.flatten(ak.mask(arrs[fields][groomed_split_idx], mask_post))        
        return idx, vals
    
    def apply_cut(self, arrs, evt_lvl):
        if f'lund_{evt_lvl}Index' not in arrs.fields:
            arrs[f'lund_{evt_lvl}Index'] = ak.local_index(arrs[f'lund_{evt_lvl}z'])
        if f'lund_{evt_lvl}num' not in arrs.fields:
            arrs[f'lund_{evt_lvl}num'] = ak.num(arrs[f'lund_{evt_lvl}z'])
        mask_pre = eval(self._convert_expression(self.cut_pre, evt_lvl))
  
#         for field in arrs.fields:
#             var = field.replace(f'lund_{evt_lvl}', '')
#             if var not self.iterable_variables:
              
        new_fields = []
        for var in self.iterable_variables:
            field = f'lund_{evt_lvl}{var}'
            if field not in arrs.fields: 
                continue
            arrs[field] = ak.mask(arrs[field], mask_pre)
            new_fields.append(field)
        return arrs[new_fields]

In [ ]:
class Groomer:
    def __init__(self, cut_groom=None, cut_pre=None, cut_post=None):
        self.descr = f'precut = {cut_pre} | groom = {cut_groom} | postcut = {cut_post}'
        self.descr_short = (f'precut = {cut_pre} | ' if cut_pre else '')+f' groom = {cut_groom} ' + (f' | postcut = {cut_post}' if cut_post else '')
        self.cut_pre = cut_pre if cut_pre is not None else 'z > -999'
        self.cut_post= cut_post  if cut_post is not None else 'z > -999'
        self.cut_groom = cut_groom
        self.iterable_variables = ['z', 'Delta', 'p_pt', 'kt', 'kappa', 'Index', 'numArr']
    
    def __str__(self):
        return self.descr
    def __repr__(self):
        return self.__str__()
    def get_label(self):
        return self.descr
    def get_label_short(self):
        return self.descr_short
    
    
    def _convert_expression(self, exp, evt_lvl, idx_str=''):
        for q in self.iterable_variables:
            exp = exp.replace(q, f"arrs[\'lund_{evt_lvl}{q}\']{idx_str}")
        return exp
    
    def groom(self, arrs, evt_lvl):
        if self.cut_groom is None:
            return ValueError
        
        if f'lund_{evt_lvl}Index' not in arrs.fields:
            arrs[f'lund_{evt_lvl}Index'] = ak.local_index(arrs[f'lund_{evt_lvl}z'])
        if f'lund_{evt_lvl}num' not in arrs.fields:
            arrs[f'lund_{evt_lvl}num'] = ak.num(arrs[f'lund_{evt_lvl}z'])
        if f'lund_{evt_lvl}numArr' not in arrs.fields:
            n = ak.num(arrs[f'lund_{evt_lvl}z']) 
            arrs[f'lund_{evt_lvl}numArr'] = n * arrs[f'lund_{evt_lvl}Index']**0 
            
        from copy import deepcopy
        arrs = deepcopy(arrs)

        mask_pre = eval(self._convert_expression(self.cut_pre, evt_lvl))   
        grooming_quantity = eval(self._convert_expression(self.cut_groom, evt_lvl))
        groomed_split_idx = ak.argmax(ak.mask(grooming_quantity, mask_pre), 
                              axis=1, keepdims=1)
        mask_post = eval(self._convert_expression(self.cut_post, evt_lvl, '[groomed_split_idx]'))
        
        idx = ak.flatten(ak.mask(groomed_split_idx, mask_post))
#         fields = [f'lund_{evt_lvl}Delta', f'lund_{evt_lvl}z', f'lund_{evt_lvl}kt', f'lund_{evt_lvl}p_pt', f'lund_{evt_lvl}kappa'
        fields = [f'lund_{evt_lvl}{var}' for var in self.iterable_variables if f'lund_{evt_lvl}{var}' in arrs.fields]
        vals = ak.flatten(ak.mask(arrs[fields][groomed_split_idx], mask_post))        
        return idx, vals
    
    
    def apply_cut(self, arrs, evt_lvl):
        if f'lund_{evt_lvl}Index' not in arrs.fields:
            arrs[f'lund_{evt_lvl}Index'] = ak.local_index(arrs[f'lund_{evt_lvl}z'])
        if f'lund_{evt_lvl}num' not in arrs.fields:
            arrs[f'lund_{evt_lvl}num'] = ak.num(arrs[f'lund_{evt_lvl}z'])
        mask_pre = eval(self._convert_expression(self.cut_pre, evt_lvl))
  
#         for field in arrs.fields:
#             var = field.replace(f'lund_{evt_lvl}', '')
#             if var not self.iterable_variables:
              
        new_fields = []
        for var in self.iterable_variables:
            field = f'lund_{evt_lvl}{var}'
            if field not in arrs.fields: 
                continue
            arrs[field] = ak.mask(arrs[field], mask_pre)
            new_fields.append(field)
        return arrs[new_fields]

In [ ]:
def convert_hist_to_weighted(h):
    storage = hist.storage.Weight()
    h_new = hist.Hist(h.axes[0], label=h.label,
                     storage=storage)
    h_new[:] = np.array([(val,var) for val,var in zip(h.values(), h.variances())])
    return h_new


def calc_ratio_1d(h1, h2, mode='ratio'):
    # for mode='ratio': implements ROOT.TH1.Divide() 
    # https://root.cern.ch/doc/master/TH1_8cxx_source.html#l02921
    
    # for symmetric errors, plot using:
    # hr.plot1d(yerr=hr.variances()**0.5)
    # for asymmetric errors, first read:
    # https://github.com/scikit-hep/mplhep/blob/36dc5a865acc2d6a8bb90704bfbab75e2ae3a65c/src/mplhep/error_estimation.py#L11-L29

    vals1 = h1.values()
    vals2 = h2.values()
    vars1 = h1.variances()
    vars2 = h2.variances()
    if mode == 'ratio':
        values = vals1/vals2
        variances = vars1 * 1./vals2**2 + vars2 * (vals1/vals2**2)**2
    elif mode == 'rel_diff':
        values = (vals1 - vals2) / vals2
        variances = vars1 * 1./vals2**2 + vars2 * (vals1/vals2**2)**2
    elif mode == 'rel_sym_diff':
        values = (vals1 - vals2) / (vals1+vals2)
        variances = vars1 * (2*vals2 / (vals1+vals2)**2)**2 + vars2 * (2*vals1 / (vals1+vals2)**2)**2
    else:
        raise ValueError(f'invalid mode: {mode}')
    hq = deepcopy(h1)
    hq[:] = np.array([(val,var) for val,var in zip(values, variances)])
    return hq   


def calc_ratio_2d(h1,h2, mode):
    # ignores errors
    hq = deepcopy(h1)
    if mode == 'rel_diff':
        hq[:,:] = (h1.values() - h2.values()) / h2.values()
    elif mode == 'rel_sym_diff':
        hq[:,:] = (h1.values() - h2.values()) / (h1.values()+h2.values())
    elif mode == 'ratio':
        hq[:,:] = h1.values() / h2.values()
    else:
        print('unknown mode')
        return
    return hq

In [ ]:


from scipy.optimize import curve_fit
def sigmoid_gen(x, a,b,c,d):
    '''
    (d-a) / (1 + np.e**(-b*(x-c)))+a
    
         a      -     b     -     c      -     d 
    left asympt - steepness - midpoint x - right asympt
    '''
    return (d-a) / (1 + np.e**(-b*(x-c)))+a


In [ ]:
def save_plot(fname, dirname=None, overwrite=True):
    fpath = Path(dirname) / Path(fname) if dirname else Path(fname)
    Path(dirname).mkdir(parents=True, exist_ok=True)
    
    if overwrite or not Path(fpath).exists():
        print('saving to ', fpath)
        plt.savefig(fpath)

In [ ]:
def save_plot(fname, dirname=None, overwrite=True):
    import pickle
    fpath = Path(dirname) / Path(fname) if dirname else Path(fname)
    Path(dirname).mkdir(parents=True, exist_ok=True)
    
    if overwrite or not Path(fpath).exists():
        print('saving to ', fpath)
        plt.savefig(fpath)
        pickle.dump(plt.gca(), open(str(fpath).replace('.png', '.pickle'), "wb"))

# Groomer

In [ ]:
cached_arrs_per_flavour_pt = dict(beauty=dict(), udsg=dict())
import gc
gc.collect()

In [ ]:
pts = [30,40,50,60,80,100,120,150,200]
selected_pt_bins = [(40,50), (80,100), (150,200)]
udsg_pt_factor = 1.0


pts_pairs = [pt for pt in zip(pts[:-1], pts[1:])] 

for ptb in selected_pt_bins:
    if ptb not in pts_pairs:
        pts_pairs.append(ptb)
        
print(pts_pairs)
for i,jet_pt in enumerate(pts_pairs):
    values = dict()
    arrs_flavour = dict()
    for flavour in ['beauty', 'udsg']:
        ### define cuts
        print(flavour, jet_pt)
        cut_jpt = f'(j_pt[:,0] > {jet_pt[0]}) & (j_pt[:,0] < {jet_pt[1]})'
#         cut_jpt = f'(j_pt[:,0] > 30) & (j_pt[:,0] < 300)'
        ### TEMP:: EQUALIZE ERAD BETWEEN UDSG AND BEAUTY !!!
        if flavour == 'udsg':
            factor = udsg_pt_factor
            cut_jpt = f'(j_pt[:,0] > {jet_pt[0]*factor}) & (j_pt[:,0] < {jet_pt[1]*factor})'        
        ### END OF TEMP
        
        arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
        if True or arrs is None:
            print(flavour, jet_pt, ' - computing')
            cut_common = f'{cut_jpt}'
            flavour_cut = flavours_cuts[flavour]
            cut = f'({cut_common}) & ({flavour_cut})'


            arrs = uproot.concatenate(paths,
                     filter_name=['j_pt', 'j_comb_pt', 
                                  'lund_z', 'lund_comb_z', 'lund_Delta', 'lund_comb_Delta', 'lund_kt', 'lund_comb_kt', 
                                  'lund_p_pt', 'lund_comb_p_pt',
                                 'lund_pair_matched_pt2'], 
                     cut=cut,
                    )
            arrs = arrs[(ak.num(arrs['lund_z']) > 0) & (ak.num(arrs['lund_comb_z']) > 0)]
            cached_arrs_per_flavour_pt[flavour][jet_pt] = arrs
            del arrs
        import gc
        gc.collect()

In [ ]:
# # groomer = Groomer(cut_pre = f'(z > 0.3 * (Delta/0.4)**(1))', cut_groom='-Index', cut_post=cut_post) # SD\n",
# # groomer = Groomer(cut_groom=f'z * (1-z) * p_pt * (Delta/0.4)**{0.1}', cut_post=cut_post) # dyn\n",
# # groomer = Groomer(cut_pre = f'kt > 0.5', cut_groom='Index', cut_post=cut_post) # late-kt\n",
# # groomer = Groomer(cut_pre = f'kt > 1', cut_groom='-Index', cut_post=cut_post) # early-kt\n",
# # groomer = Groomer(cut_groom=f'z * (Delta/0.4)**{2}', cut_post=cut_post) # min-tf\n",
# # groomer = Groomer(cut_groom=f'-z * (Delta/0.4)**{2}', cut_post=cut_post) # max-tf\n",
# groomer = Groomer(cut_pre = f'(z > {par}) & (kt > {par*50})', cut_groom='Index', cut_post=cut_post))\n",
# groomer = Groomer(cut_pre = f'(z > 0.0 * (Delta/0.4)**(0)) & (kt > 0)', cut_groom='z', cut_post=cut_post)\n",
# groomer = Groomer(cut_post = f'(kt > 0.2)', cut_groom=f'z * (1-z) * p_pt * (Delta/0.4)**{0.01}')\n",
# groomer = Groomer(cut_groom=f'z * (1-z) * p_pt * (Delta/0.4)**{2.0}', cut_post='(p_pt > 40) & (p_pt < 50)')\n",
# groomer = Groomer(cut_pre = f'kt > 0.0', cut_groom='z', cut_post=cut_post)\n",
# groomer = Groomer(cut_groom=f'z * (1-z) * p_pt * (Delta/0.4)**{0.1}', cut_post=cut_post)\n",
# groomer = Groomer(cut_groom=f'z * p_pt', cut_post=cut_post)\n",
# groomer = Groomer(cut_pre = f'(z > 0.3 * (Delta/0.4)**(1)) & (kt > 0.5)', cut_groom='-Index', cut_post=cut_post)\n",
# groomer = Groomer(cut_pre = f'(z > 0.2 * (Delta/0.4)**(0)) & (kt > 0.0)', cut_groom='Index', cut_post=cut_post)\n",


In [ ]:
udsg_pt_factor_str = 'udsgPtF-' + str(udsg_pt_factor).replace('.', 'p')

In [ ]:
cut_post=None

# cut_z = 0.2
# cut_kt = 0.1
# groomer_const = Groomer(cut_pre=f'(kt > {cut_kt})',cut_groom='z/Delta', cut_post=cut_post)
# cut_str = groomer_const.cut_pre.replace('(', '').replace(')', '').replace('>', '').replace('<', '').replace('&', '_').replace('|', '_OR_').replace(' ', '').replace('.', '')

# cut_kt = 0.1
# groomer_const = Groomer(cut_pre=f'(kt > {cut_kt[0]}) & (kt < {cut_kt[1]})',cut_groom='z', cut_post=cut_post)
# cut_str = groomer_const.cut_pre.replace('(', '').replace(')', '').replace('>', '').replace('<', '').replace('&', '_').replace('|', '_OR_').replace(' ', '').replace('.', '')


# groomer_const = Groomer(cut_pre = f'(z > 0.1 * (Delta/0.4)**(0))', cut_groom='z', cut_post=cut_post)


# dirname = f'plots_thesis_prep/zOverTheta_{cut_str}__{udsg_pt_factor_str}'
# dirname = f'plots_thesis_prep/SD_0_02__{udsg_pt_factor_str}'
# dirname = f'plots_thesis_prep/test__{udsg_pt_factor_str}'




# groomer_const = Groomer(cut_groom=f'z * (1-z) * p_pt * (Delta/0.4)**{0.01}', cut_post=cut_post)
# dirname = f'thesis-plots/dchic/v1/DyG_001__{udsg_pt_factor_str}'

# groomer_const = Groomer(cut_pre=None,cut_groom='z', cut_post=cut_post)
# dirname = f'thesis-plots/dchic/v1/max-z__{udsg_pt_factor_str}'

# groomer_const = Groomer(cut_pre = f'(z > 0.2 * (Delta/0.4)**(0))', cut_groom='-Index', cut_post=cut_post) # SD,
# dirname = f'thesis-plots/dchic/v1/SD_0_02__{udsg_pt_factor_str}'

# groomer_const = Groomer(cut_pre=f'(z > 0.2 * (Delta/0.4)**(0))',cut_groom='Index', cut_post=cut_post)
# dirname = f'thesis-plots/dchic/v1/invSD_0_02__{udsg_pt_factor_str}'

groomer_const = Groomer(cut_pre=f'(kt > 0.1)',cut_groom='z/Delta', cut_post=cut_post)
dirname = f'thesis-plots/dchic/v1/zOverDelta_kt01__{udsg_pt_factor_str}'


print(dirname)
Path(dirname).mkdir(parents=True, exist_ok=True)
with open(Path(dirname) / Path('groomer_descr.txt'), 'w') as f:
    f.write(groomer_const.descr)
    

In [ ]:
# space = [-2,-1,0,1,2]
# n = 0
# max_sum_of_powers = 3
# q_dict = dict()
# for a in space:
#     for b in space:
#         for c in space:
#             if abs(a)+abs(b)+abs(c) > max_sum_of_powers:
#                 continue
#             if a==b and b==c: 
#                 continue
#             if all([abs(power) == 2 or power == 0 for power in (a,b,c)]):
#                 continue
# #             print(a,b,c)
# #             n += 1
# #             groomer = Groomer(cut_pre = None, cut_groom=f'kt**{a} * z**{b} * Delta**{c}')
# #             print(groomer)

#             arrs = cached_arrs_per_flavour_pt['udsg'].get((30,200), None) 
#             lvl = ''
#             vals = arrs[f'lund_{lvl}kt']**a * arrs[f'lund_{lvl}z']**b * arrs[f'lund_{lvl}Delta']**c
#             vals_udsg = ak.flatten(vals).to_numpy()
            
#             arrs = cached_arrs_per_flavour_pt['beauty'].get((30,200), None) 
#             lvl = ''
#             vals = arrs[f'lund_{lvl}kt']**a * arrs[f'lund_{lvl}z']**b * arrs[f'lund_{lvl}Delta']**c
#             vals_b = ak.flatten(vals).to_numpy()
            
#             vals_all = np.hstack([vals_b, vals_udsg])

#             qs = [0.001, 0.01, 0.03, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.97, 0.99, 0.999]
#             qvs = np.quantile(vals_all, qs)
#             print(qvs)
#             q_dict[f'{a}{b}{c}'] = dict()
#             for q,qv in zip(qs,qvs):
#                 q_dict[f'{a}{b}{c}'][q] = qv
# # print('n = ',n)

In [ ]:
# import json

# with open('quantiles.json', 'w') as f:
#     json.dump(q_dict, f)

In [ ]:
# import json
# with open('quantiles.json') as f:
#     d=json.load(f)
# print(d)

In [ ]:
for jet_pt in selected_pt_bins:
    binning_kt = 60,-5.5,3.9
    binning_delta = 51,0.8,5.9


    h_b_py = Hist(hist.axis.Regular(*binning_delta, name='Delta', label=LABELS['lnOneOverTheta']), 
               hist.axis.Regular(*binning_kt, name='kt', label=LABELS['lnkt']),
              storage=hist.storage.Double())
    h_lf_py = Hist(hist.axis.Regular(*binning_delta, name='Delta', label=LABELS['lnOneOverTheta']), 
               hist.axis.Regular(*binning_kt, name='kt', label=LABELS['lnkt']),
              storage=hist.storage.Double())

    h_b_comb = Hist(hist.axis.Regular(*binning_delta, name='Delta', label=LABELS['lnOneOverTheta']), 
               hist.axis.Regular(*binning_kt, name='kt', label=LABELS['lnkt']),
              storage=hist.storage.Double())
    h_lf_comb = Hist(hist.axis.Regular(*binning_delta, name='Delta', label=LABELS['lnOneOverTheta']), 
               hist.axis.Regular(*binning_kt, name='kt', label=LABELS['lnkt']),
              storage=hist.storage.Double())



    hs = [h_b_py, h_lf_py, h_b_comb, h_lf_comb]

    fig,axes = plt.subplots(ncols=2, nrows=2, figsize=(10,6), sharex=True, sharey=True)
#     plt.tight_layout(pad=2.5)
    fig.subplots_adjust(hspace=0, wspace=0, left=0.07, right=1, top=0.9)
    for h,flavour,lvl,ax in zip(hs,['beauty', 'udsg',]*2, ['', '', 'comb_', 'comb_'], axes.flatten()):

        arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
        if arrs is None:
            print(flavour, jet_pt, ' is none')
            raise ValueError('not found in cache')

        __, arrs = groomer_const.groom(arrs, lvl)
        deltas = ak.flatten(arrs[f'lund_{lvl}Delta'])
        kts = ak.flatten(arrs[f'lund_{lvl}kt'])

#         arrs = groomer_const.apply_cut(arrs, lvl)
#         deltas = ak.flatten(ak.flatten(arrs[f'lund_{lvl}Delta']), axis=0)
#         kts = ak.flatten(ak.flatten(arrs[f'lund_{lvl}kt']), axis=0)
        
        
        h.fill(np.log(1/deltas), np.log(kts))

        kws_hist2d = dict(cmap='viridis',
#                           cbarpad=0.1, cbarsize='7%',
                          cbar=False,
    #                       cmin=1e-5,
    #                       vmin=1e-3,vmax=3e-2
                          norm=matplotlib.colors.LogNorm(vmin=2e-4, vmax=1e-2)
                          
                         )
        #norm to unity
        h[:,:] = h.values() / np.sum(h.values())

        # norm to Njets
    #     h[:,:] = h.values() / len(arrs)


        art = mplhep.hist2dplot(h, ax=ax,  **kws_hist2d)
#         ax.set_title(f'       {flavour} {lvl}')
        flavour_lvl_caption = r'$\it{b}$-jets' if flavour == 'beauty' else r'$\it{udsg}$-jets'
        flavour_lvl_caption += ' ' + ('embedded' if 'comb' in lvl else 'PYTHIA')
        ax.text(0.98, 0.98, flavour_lvl_caption, 
                transform=ax.transAxes, ha='right', va='top', fontsize='large')

        plt.grid(ls=':')
    cbar = fig.colorbar(art.pcolormesh, ax=axes[:, :], 
                 pad=0.03,
                 aspect=30)
    cbar.set_label(label=r'$\frac{1}{\rho} \; \frac{d\rho}{d\,\ln 1/\theta \;\; d\,\ln k_{\rm T}}$', fontsize='xx-large')
    for ax in axes.flatten():
        ax.set_xlabel(LABELS['lnOneOverTheta'] if ax in axes[-1,:] else '', fontsize='large')        
        ax.set_ylabel(LABELS['lnkt'] if ax in axes[:,0] else '', fontsize='large')

#     pt_caption = f'{jet_pt[0]:.0f} < ' + LABELS['ptjet'] + f'< {jet_pt[1]:.0f}' + LABELS['gevc']
    pt_caption = f"{jet_pt[0]:.0f} < {LABELS['ptjet']} < {jet_pt[1]:.0f} {LABELS['gevc']}"
    axes[0,0].text(1.1, 1.05, pt_caption, transform=axes[0,0].transAxes, va='bottom', ha='center', fontsize='x-large')
    save_plot(f'lp_pt{jet_pt[0]}-{jet_pt[1]}.png',dirname)

    ###
    ###
    ###
    
    
    from copy import deepcopy

    mode = 'rel_sym_diff'
    max_rel_diff = 10
    if mode == 'ratio':
        norm = matplotlib.colors.LogNorm(vmin=1./max_rel_diff, vmax=max_rel_diff)
    elif mode == 'rel_diff' or mode == 'rel_sym_diff':
        norm = matplotlib.colors.Normalize(vmin=-1, vmax=1)

    # norm = matplotlib.colors.SymLogNorm(linthresh=0.03, linscale=0.03, vmin=-1.0, vmax=1.0, base=10)
    # norm = matplotlib.colors.Normalize(vmin=-1.5, vmax=1.5)
    kws_hist2d = dict(cmap='coolwarm',
#                       cbarpad=0.1, cbarsize='7%',
                      cbar=False,
                      norm=norm
                     )
    fig,axes = plt.subplots(ncols=2, nrows=2, figsize=(10,6), sharex=True, sharey=True)
    fig.subplots_adjust(hspace=0, wspace=0, left=0.07, right=1, top=0.9)


    # PY: b/lf
    hr = calc_ratio_2d(h_b_py , h_lf_py, mode)
    mplhep.hist2dplot(hr, ax=axes[0,0], **kws_hist2d)
#     axes[0,0].set_title(f'{mode} b/lf (PY)')

    # comb: b/lf
    hr = calc_ratio_2d(h_b_comb , h_lf_comb, mode)
    mplhep.hist2dplot(hr, ax=axes[0,1], **kws_hist2d)
#     axes[0,1].set_title(f'{mode} b/lf (comb)')

    # b: comb/PY
    hr = calc_ratio_2d(h_b_comb , h_b_py, mode)
    mplhep.hist2dplot(hr, ax=axes[1,0], **kws_hist2d)
#     axes[1,0].set_title(f'{mode} comb/PY (b)')


    # lf: comb/PY
    hr = calc_ratio_2d(h_lf_comb , h_lf_py, mode)
    art = mplhep.hist2dplot(hr, ax=axes[1,1], **kws_hist2d)
#     axes[1,1].set_title(f'{mode} comb/PY (lf)')

    cbar = fig.colorbar(art.pcolormesh, ax=axes[:, :], 
                 pad=0.03,
                 aspect=30)
    cbar.set_label(label=r'$Q$', fontsize='x-large')
    for ax in axes.flatten():
        ax.set_xlabel(LABELS['lnOneOverTheta'] if ax in axes[-1,:] else '', fontsize='large')        
        ax.set_ylabel(LABELS['lnkt'] if ax in axes[:,0] else '', fontsize='large')
    pt_caption = f"{jet_pt[0]:.0f} < {LABELS['ptjet']} < {jet_pt[1]:.0f} {LABELS['gevc']}"

    axes[0,0].text(1.1, 1.05, pt_caption, transform=axes[0,0].transAxes, va='bottom', ha='center', fontsize='x-large')
    
    axes[0,0].text(0.98, 0.98, 'PYTHIA   ' + LABELS['Q_b_udsg'], transform=axes[0,0].transAxes, ha='right', va='top', fontsize='large')
    axes[0,1].text(0.98, 0.98, 'embedded   ' + LABELS['Q_b_udsg'], transform=axes[0,1].transAxes, ha='right', va='top', fontsize='large')
    axes[1,0].text(0.98, 0.98, r'$\it{b}$-jets   ' + LABELS['Q_comb_py'], transform=axes[1,0].transAxes, ha='right', va='top', fontsize='large')
    axes[1,1].text(0.98, 0.98, r'$\it{udsg}$-jets   ' + LABELS['Q_comb_py'], transform=axes[1,1].transAxes, ha='right', va='top', fontsize='large')

    
    for ax in axes.flatten():
    #     plt.sca(ax)
        ax.grid(ls=':')

    save_plot(f'lp_ratios_pt{jet_pt[0]}-{jet_pt[1]}.png',dirname)

In [ ]:
fig,axes = plt.subplots(figsize=(12,5), ncols=4)
for flavour in ['beauty', 'udsg']:
        pur = []
        eff = []
        eff_py, eff_comb = [], []
        pt = []
        aver_pur = []
        for jet_pt in zip(pts[:-1], pts[1:]):



            arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
            if arrs is None:
                print(flavour, jet_pt, ' is none')
                raise ValueError('not found in cache')



            idx_py,vals_py = groomer_const.groom(arrs, '')
            idx_comb,vals_comb = groomer_const.groom(arrs, 'comb_')
            idx_pair = (idx_py * arrs['lund_comb_num'] + idx_comb)
            v = arrs['lund_pair_matched_pt2'][ak.singletons(idx_pair)]
            pur_cur = np.sum(v > 0.5)/len(ak.flatten(v))
            eff_cur = len(ak.flatten(v))/len(v)
            vv = ak.flatten(v)
            aver_pur_cur = np.mean(vv[vv > 0.5])
            print(pur_cur, eff_cur)
            pur.append(pur_cur)
            eff.append(eff_cur)
            eff_py.append(np.sum(idx_py > -1)/len(idx_py))
            eff_comb.append(np.sum(idx_comb > -1)/len(idx_comb))
            pt.append(np.mean(jet_pt))
            aver_pur.append(aver_pur_cur)
        axes[0].plot(pt,pur, 'o-' if flavour == 'beauty' else 'o--', label=flavour)
        axes[1].plot(pt,eff_py, '.-' if flavour == 'beauty' else '.--', label=flavour)
        axes[2].plot(pt,eff_comb, '.-' if flavour == 'beauty' else '.--', label=flavour)
        axes[3].plot(pt,eff, '.-' if flavour == 'beauty' else '.--', label=flavour)
#         plt.plot(pt,aver_pur, 'o--', label=flavour+'__'+str(par))
for ax in axes:
    ax.legend()
    ax.grid()
    ax.set_ylim(0,1.05)
axes[0].set_title('purity')
axes[1].set_title('eff PY')
axes[2].set_title('eff comb')
axes[3].set_title('eff both')
save_plot('pur_eff_vs_pt_old.png',dirname)

In [ ]:
from matplotlib import rc,rcdefaults, rcParams
rcdefaults()
rc('axes',labelsize='large')
rc('legend', frameon=False, fontsize='medium')

fig,axes = plt.subplots(figsize=(10,4), ncols=2)
for flavour in ['beauty', 'udsg']:
        pur = []
        eff = []
        eff_py, eff_comb = [], []
        eff_py_if_comb = []
        pt = []
        aver_pur = []
        for jet_pt in zip(pts[:-1], pts[1:]):

            arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
            if arrs is None:
                print(flavour, jet_pt, ' is none')
                raise ValueError('not found in cache')

            idx_py,vals_py = groomer_const.groom(arrs, '')
            idx_comb,vals_comb = groomer_const.groom(arrs, 'comb_')
            idx_pair = (idx_py * arrs['lund_comb_num'] + idx_comb)
            v = arrs['lund_pair_matched_pt2'][ak.singletons(idx_pair)]
            pur_cur = np.sum(v > 0.5)/len(ak.flatten(v))
            eff_cur = len(ak.flatten(v))/len(v)
            vv = ak.flatten(v)
            aver_pur_cur = np.mean(vv[vv > 0.5])
            print(pur_cur, eff_cur)
            pur.append(pur_cur)
            eff.append(eff_cur)
            eff_py.append(np.sum(idx_py > -1)/len(idx_py))
            eff_comb.append(np.sum(idx_comb > -1)/len(idx_comb))
            eff_py_if_comb.append(np.sum((idx_py > -1) & (idx_comb > -1))/np.sum(idx_comb > -1))
            pt.append(np.mean(jet_pt))
            aver_pur.append(aver_pur_cur)
        color = '#FF0000' if flavour == 'beauty' else '#000080'
        marker = 'D' if flavour == 'beauty' else 's'
        label_core = r'$\it{b}$-jets' if flavour =='beauty' else '$\it{udsg}$-jets'
        axes[0].plot(pt,pur, marker=marker, ms=4, ls='-', label=label_core, color=color)
        axes[1].plot(pt,eff_py, marker=marker, ms=6, ls='-', label=label_core + r' $\epsilon_{\rm g}^{\rm \;vac}$', color=brighter(color))
        axes[1].plot(pt,eff_comb, marker=marker, ms=6, ls=':', lw=2, label=label_core + r' $\epsilon_{\rm g}^{\rm \;bckg}$', color=darker(color), fillstyle='none')
        axes[1].plot(pt,eff_py_if_comb, marker="d" if flavour == 'beauty' else 'P', ms=7, lw=1, ls='--', label=label_core + r' $\epsilon_{\rm g}^{\rm \;vac\;|\;bckg}$', color=(color))
        
for ax in axes:
    ax.legend(frameon=False, fontsize='medium', ncol=2, handletextpad=0.1, columnspacing=2)
#     ax.grid(ls=':')
    ax.set_ylim(0,1.5)
#     ax.set_xlim(20,200)
    ax.set_xticks([20, 40, 60, 80, 100, 120, 140, 160, 180, 200])
    ax.set_xlabel(f"{LABELS['ptjet']} ({LABELS['gevc']})")
    ax.axhline(1, ls=':', color='darkgray')
axes[0].set_ylabel('subleading prong purity')
axes[1].set_ylabel(r'grooming efficiency  $\epsilon_{\rm g}$')

plt.tight_layout(w_pad=2)
save_plot('pur_eff_vs_pt.png',dirname)
rcdefaults()

In [ ]:
binning_rel_diff=(50,-5,5)

var='lund_Delta'
fail_dummy_value = None
binning = None


default_binnings = {'lund_z':(14,-0.1,0.6),
                   'lund_Delta':(12,-0.1,0.5)}
default_fail_dummy_values = {'lund_z':-0.06, 'lund_Delta':-0.06} # should land in first bins for proper eff calc.

if binning is None:
    binning = default_binnings[var]
if fail_dummy_value is None:
    fail_dummy_value = default_fail_dummy_values[var]

        
for jet_pt in selected_pt_bins:

        
    # idx_colname = f'gidx_{groomer}'
    # idx_colname_comb = idx_colname.replace('gidx_', 'gidx_comb_')
    var_comb = var.replace('lund_', 'lund_comb_')

    ### define cuts
    cut_jpt = f'(j_pt[:,0] > {jet_pt[0]}) & (j_pt[:,0] < {jet_pt[1]})'
    # cut_has_splits = f'({idx_colname}[:,0] >= 0) & ({idx_colname_comb}[:,0] >= 0)'
    cut_common = f'{cut_jpt}'
    #                 print(cut_common.replace('&', '\n&').replace('|','|\n'))

    label_py = '$'+var2str(var)+'^{PYTHIA}$'
    label_comb = '$'+var2str(var)+'^{PY+bckg}$'
    # label_rel_diff = '$('+label_comb + ' - ' + label_py + ') / ' + label_py + '$'
    label_rel_diff = rf'$({label_comb.replace("$","")} - {label_py.replace("$","")}) / {label_py.replace("$","")}$'
    label_mean_diff = rf'$\langle{label_rel_diff.replace("$","")}\rangle$'
    label_std_diff = rf'$\sigma({label_rel_diff.replace("$","")})$'

    h_b = Hist(hist.axis.Regular(*binning, name=var, label=label_py), 
               hist.axis.Regular(*binning, name=var_comb, label=label_comb),
              storage=hist.storage.Double())
    h_lf = Hist(hist.axis.Regular(*binning, name=label_py), hist.axis.Regular(*binning, name=label_comb))

    hs = [h_lf, h_b]
    fig,axes = plt.subplots(ncols=3, nrows=2, figsize=(12,7))
    effs_txt = ''
    ### get 2D histo per flavour: PY+comb vs PY
    for h,flavour in zip(hs,['udsg', 'beauty']):


        arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
        if arrs is None:
            print(flavour, jet_pt, ' is none')
            raise ValueError('not found in cache')





        idx_py,vals_py = groomer_const.groom(arrs, '')
        idx_comb,vals_comb = groomer_const.groom(arrs, 'comb_')
        vals_py = ak.fill_none(vals_py[var], [fail_dummy_value], axis=0)
        vals_comb = ak.fill_none(vals_comb[var_comb], [fail_dummy_value], axis=0)

        vals_py = ak.flatten(vals_py)
        vals_comb = ak.flatten(vals_comb)





        h.fill(vals_py, vals_comb)

        h[:,:] = np.nan_to_num(h.values())
        h_py   = h.project(0)
        h_comb = h.project(1)

        kws_hist1d = dict(histtype='errorbar', markersize=6,
                          elinewidth=1,
                          marker=('s' if flavour == 'udsg' else 'o'),
                          ecolor=colors[flavour],mec=colors[flavour],
                         )

        binwidth=h_py.axes.widths[0][0]
        integ = np.sum(h_py.values())
        mplhep.histplot(h_py, binwnorm=binwidth/integ, ax=axes[0,2],xerr=True, **kws_hist1d, mfc=colors[flavour], label=flavour+' PYTHIA')
        binwidth=h_comb.axes.widths[0][0]
        integ = np.sum(h_comb.values())
        mplhep.histplot(h_comb, binwnorm=binwidth/integ, ax=axes[0,2], xerr=True, **kws_hist1d, mfc='white', label=flavour+' PY+bckg')


        ### rel diff
        vals_rel_diff = (vals_comb - vals_py) / vals_py
        mask_only_valid = (vals_py != fail_dummy_value) & (vals_comb != fail_dummy_value)
    #     vals_rel_diff = vals_rel_diff[mask_only_valid]
    #     mplhep.histplot(np.histogram(vals_rel_diff, bins=binning_rel_diff), 
    #                     binwnorm=(binning_rel_diff[1]-binning_rel_diff[0])/len(vals_rel_diff),
    #                     ax=axes[1,0], mfc=colors[flavour], **kws_hist1d)

        h_rel_diff_2d = Hist(hist.axis.Regular(*binning, name=label_py),
                             hist.axis.Regular(*binning_rel_diff, name=label_rel_diff), 
                             )
        h_rel_diff_2d.fill(vals_py[mask_only_valid], vals_rel_diff[mask_only_valid])
        h_rel_diff = h_rel_diff_2d.project(1)
        mplhep.histplot(h_rel_diff, yerr=np.sqrt(h_rel_diff.variances()), 
                        binwnorm=h_rel_diff.axes.widths[0][0]/len(vals_rel_diff[mask_only_valid]),
                        ax=axes[1,0], mfc=colors[flavour], label=flavour, **kws_hist1d)

        hprof = h_rel_diff_2d.profile(1)
        vals_means = hprof.values()
        vals_stds = np.sqrt(hprof.view()['_sum_of_deltas_squared'] / hprof.view()['count'])
    #     h_rel_diff_2d.profile(1).plot()

        ### JES
        h_means = h_py.copy()
        h_means[:] = vals_means
        mplhep.histplot(h_means, yerr=0, ax=axes[1,1], linestyle='-', mfc=colors[flavour], color=colors[flavour], label=flavour, **kws_hist1d)
        ### JER
        h_stds = h_py.copy()
        h_stds[:] = vals_stds
        mplhep.histplot(h_stds, yerr=0, ax=axes[1,2], linestyle='-', mfc=colors[flavour], color=colors[flavour], label=flavour, **kws_hist1d)

        ### effs
        n_tot = np.sum(h[:,:].values())
        n_py = np.sum(h[1:,:].values())
        n_comb = np.sum(h[:,1:].values())
        n_both = np.sum(h[1:,1:].values())

        effs_txt += ('\n' if len(effs_txt) else '') + '$\epsilon_{' + (flavour if len(flavour) < 6 else flavour[0]) + '}' +f': PY={n_py/n_tot:.3f}, \; PY\!+\!bckg={n_comb/n_tot:.3f}, \; &={n_both/n_tot:.3f}$'



    axes[0,0].axvline(0, c='gray', ls='--')
    axes[0,0].axhline(0, c='gray', ls='--')
    axes[0,1].axvline(0, c='gray', ls='--')
    axes[0,1].axhline(0, c='gray', ls='--')


    axes[0,2].text(0.5, 1.02, effs_txt,
                  transform=axes[0,2].transAxes,
                  horizontalalignment='center',
                  verticalalignment='bottom',
                  fontsize=9,
    )
    axes[0,2].axvline(0, c='gray', ls='--')
    axes[0,2].set_yscale('log')
    axes[0,2].set_ylim(1e-3,2)
    axes[0,2].set_ylabel('proba')
    axes[0,2].set_xlabel(f'{label_py} or {label_comb}')
    axes[0,2].legend(framealpha=0.3)

    axes[1,0].axvline(0, c='gray', ls='--')
    axes[1,0].set_yscale('log')
    axes[1,0].set_ylabel('proba')
    axes[1,0].set_ylim(bottom=1e-3)
    axes[1,0].legend()

    axes[1,1].axhline(0, c='gray', ls='--')
    axes[1,1].set_ylabel(label_mean_diff)
    axes[1,1].set_ylim(-2,2)
    axes[1,1].legend()

    axes[1,2].set_ylabel(label_std_diff)
    axes[1,2].set_ylim(0,3)
    axes[1,2].legend()

    for ax in axes.flatten():
        if ax.get_legend(): mplhep.plot.yscale_legend(ax)

    ### plotting
    # kws_hist2d = dict(cmap='viridis')
    # h_lf.plot2d(ax=axes[0,0],  **kws_hist2d)
    # h_b.plot2d(ax=axes[0,1],  **kws_hist2d)
    kws_hist2d = dict(cmap='viridis',
                      cbarpad=0.1, cbarsize='7%',
    #                   cmin=0.005,
                      norm=matplotlib.colors.LogNorm(vmin=0.02, vmax=1)
                     )
    # norm per column (aka truth bin)
    for h in [h_lf, h_b]:
        for i_col in range(h_lf.shape[0]):
            h[i_col,:] = h[i_col,:].values() / np.sum(h[i_col,:].values())
    #         h[:,i_col] = h[:,i_col].values() / np.sum(h[:,i_col].values())

    # norm to total integral
    # for h in [h_lf, h_b]:
    #     h[:,:] = h.values() / np.sum(h.values())

    mplhep.hist2dplot(h_lf, ax=axes[0,0],  **kws_hist2d)
    mplhep.hist2dplot(h_b, ax=axes[0,1],  **kws_hist2d)
    axes[0,0].set_title('udsg')
    axes[0,1].set_title('beauty')


    axes[1,1].text(0.5, 1.02,
    #               '$p_{T,jet}^{PYTHIA}$ = ' + f'{jet_pt[0]}-{jet_pt[1]} Gev/$c$, ' + '$k_{T}$ > ' + f'{kt_cut}, ' + '$z$ > ' + f'{z_cut}'+r'\n\textbf{'+ f'{groomer2str(groomer)}'+'}',
    #               f'' + '$p_{T,jet}^{PYTHIA}$ = ' + f'{jet_pt[0]}-{jet_pt[1]} Gev/$c$, ' + '$k_{T}$ > ' + f'{kt_cut}, ' + '$z$ > ' + f'{z_cut}',
                   '',
                  transform=axes[1,1].transAxes,
                  horizontalalignment='center',
                  verticalalignment='bottom',
                  fontsize=8,
    )


    plt.tight_layout()

    save_plot(f'RM_pt{jet_pt[0]}-{jet_pt[1]}.png',dirname)

In [ ]:
for jet_pt in selected_pt_bins:
    values = dict()
    arrs_flavour = dict()

    for flavour in ['beauty', 'udsg']:


        arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
        if arrs is None:
            print(flavour, jet_pt, ' is none')
            raise ValueError('not found in cache')

        idx_py,vals_py = groomer_const.groom(arrs, '')
        idx_comb,vals_comb = groomer_const.groom(arrs, 'comb_')

        values[flavour+'_py'] = vals_py
        values[flavour+'_comb'] = vals_comb
        arrs_flavour[flavour] = arrs


    fig,ax = plt.subplots(figsize=(5,4))
    for k in values:
        vv = values[k]['lund_p_pt' if 'py' in k else 'lund_comb_p_pt']
        vv = ak.flatten(vv).to_numpy()
        qq = np.quantile(vv, [0.1,0.5,0.9])
        qq = [np.round(x,1) for x in qq]
        print(k,qq)
        
        kws = deepcopy(stylings[k])
        kws.pop('marker')
        if 'fillstyle' in kws: 
            kws.pop('fillstyle')
            
        flavour_lvl_caption = r'$\it{b}$-jets' if 'beauty' in k else r'$\it{udsg}$-jets'
        flavour_lvl_caption += ' ' + ('embedded' if 'comb' in k else 'PYTHIA')
        ax.hist(vv, bins=np.arange(0,200,2), histtype='step', density=1, 
                 label=flavour_lvl_caption, 
#                  ls='--' if 'comb' in k else '-',
                 **kws
                )

    pt_caption = f"{jet_pt[0]:.0f} < {LABELS['ptjet']} < {jet_pt[1]:.0f} {LABELS['gevc']}"
    ax.text(0.03, 0.7, pt_caption, transform=ax.transAxes, va='top', ha='left')
    
    ax.set_xlim(0,jet_pt[1]*1.3)
    ax.set_xticks([x for x in [0,20, 40, 60, 80, 100, 120, 140, 160, 180, 200, 220,240,260,280,300] if x < jet_pt[1]*1.3])
    ax.set_ylim(top=ax.get_ylim()[1]*1.3)
    ax.set_xlabel(f"{LABELS['erad']} ({LABELS['gev']})", fontsize='large')
    ax.set_ylabel(r"$\frac{1}{\sigma}\; \frac{{\rm d} \sigma}{{\rm d} E_{\rm\,rad}}$", fontsize='x-large')
    ax.legend(frameon=False, fontsize='small', labelspacing=0.5,  
#               bbox_to_anchor=(0, 0, 1, 0.9),
#               loc='upper left',
              ncol=2,
             )
    
    plt.tight_layout()
    save_plot(f'erad_distr_pt{jet_pt[0]}-{jet_pt[1]}.png',dirname)


    plt.figure()
    for k in values:
        # gets jet_pt on groomed jets
        mm = ~values[k]['lund_p_pt' if 'py' in k else 'lund_comb_p_pt'].to_numpy().mask
        jj = arrs_flavour[k.split('_')[0]]['j_pt'][mm]
    #     jj = arrs_flavour[k.split('_')[0]]['j_pt' if 'py' in k else 'j_comb_pt'][mm]
        jj = jj.to_numpy()
        qq = np.quantile(jj, [0.1,0.5,0.9])
        qq = [np.round(x,1) for x in qq]
        print(k,qq)

        plt.hist(jj, bins=np.arange(0,250,1), histtype='step', density=1, label=k, ls='--' if 'comb' in k else '-')
    plt.legend(loc='upper left')

    save_plot(f'pt_distr_pt{jet_pt[0]}-{jet_pt[1]}.png',dirname)

In [ ]:
# * theta distr:

# * fits:
# 	- labels: x: ln(1/theta), y: Q(b,udsg)
# 	- legend: only theta crit, take colors from colordict {PYTHIA, embedded}



mode = 'rel_sym_diff'


# colors = {'beauty': 'r', 'udsg':'navy', 'dc_py':'k', 'dc_comb':'gray'}
# lses = {'comb':':', 'py':'-', 'ratio':'-'}

# def get_prop(prop_dict, label):
#     for k in prop_dict.keys():
#         if k in label:
#             return prop_dict[k]





fit_params = dict(py=dict(), comb=dict())
erads = dict()

fits_ncols = int(np.ceil((len(pts)-1)/2))
fits_nrows = 2
fig_fits, axes_fits = plt.subplots(nrows=fits_nrows, ncols=fits_ncols, figsize=(10,6),
                                  sharex=True, sharey=True)
fig_fits.subplots_adjust(hspace=0, wspace=0, left=0.08, right=0.98, top=0.98)

for i,jet_pt in enumerate(zip(pts[:-1], pts[1:])):
    values = dict()
    arrs_flavour = dict()
    for flavour in ['beauty', 'udsg']:
        arrs = cached_arrs_per_flavour_pt[flavour].get(jet_pt, None)
        if arrs is None:
            print(flavour, jet_pt, ' is none')
            raise ValueError('not found in cache')

        idx_py,vals_py = groomer_const.groom(arrs, '')
        idx_comb,vals_comb = groomer_const.groom(arrs, 'comb_')
        
        
        values[flavour+'_py'] = vals_py
        values[flavour+'_comb'] = vals_comb
        arrs_flavour[flavour] = arrs

    for k in values:
        flavour, py_or_comb = k.split('_')
        vv = values[k]['lund_p_pt' if 'py' in k else 'lund_comb_p_pt']
        vv = ak.flatten(vv).to_numpy()
        qs = [0.1,0.25,0.5,0.75,0.9]
        qv = np.quantile(vv, qs)
        erads[py_or_comb] = erads.get(py_or_comb, dict())
        erads[py_or_comb][flavour] = erads[py_or_comb].get(flavour, dict())
#         erads[py_or_comb][flavour][jet_pt] = erads[py_or_comb][flavour].get(jet_pt, dict())
        for qs,qv in zip(qs,qv):
            qss = f'q{int(qs*100)}'
            erads[py_or_comb][flavour][qss] = erads[py_or_comb][flavour].get(qss, dict())
            erads[py_or_comb][flavour][qss][jet_pt] = qv
    #######################

    binning = 20,0.9,5
    
    h_b_py = Hist(hist.axis.Regular(*binning, name='lund_Delta', label=LABELS['lnOneOverTheta']), 
                  storage=hist.storage.Double(), 
                  label='beauty_py')
    h_lf_py = Hist(hist.axis.Regular(*binning, name='lund_Delta', label=LABELS['lnOneOverTheta']), 
                   storage=hist.storage.Double(), 
                   label='udsg_py')
    h_b_comb = Hist(hist.axis.Regular(*binning, name='lund_Delta', label=LABELS['lnOneOverTheta']), 
                    storage=hist.storage.Double(), 
                    label='beauty_comb')
    h_lf_comb = Hist(hist.axis.Regular(*binning, name='lund_Delta', label=LABELS['lnOneOverTheta']), 
                     storage=hist.storage.Double(), 
                     label='udsg_comb')

    hs = (h_b_py, h_lf_py, h_b_comb, h_lf_comb)
    hs = [convert_hist_to_weighted(h) for h in hs]
    (h_b_py, h_lf_py, h_b_comb, h_lf_comb) = hs

    for h in hs:
        lvl = '' if 'comb' not in h.label else 'comb_'
        vals = ak.flatten(values[h.label][f'lund_{lvl}Delta'])
    #     h.fill(vals)
        h.fill(np.log(1/vals))
    #     h /= len(values[h.label]) # norm per jet
        h /= h.sum()['value'] # self-norm 








    ########################



    fig,axes = plt.subplots(figsize=(4,6), nrows=3, 
                            gridspec_kw={'height_ratios':[2,1,1]},
                           sharex=True)
    fig.subplots_adjust(hspace=0, left=0.22, top=0.98, right=0.95)
#     fig.suptitle(str(jet_pt))

    ax = axes[0]
    for h in hs:
#         kws = dict(
#                 color=get_prop(colors, h.label), 
#                 ls=get_prop(lses, h.label),
#         )
        from copy import deepcopy
        kws = deepcopy(stylings[h.label])
        kws.pop('marker')
        if 'fillstyle' in kws: kws.pop('fillstyle')
        print(kws)
        h.plot1d(ax=ax, **kws)
        lab2lab = {'beauty_py': '$\it{b}$-jets PYTHIA',
                   'udsg_py': '$\it{udsg}$-jets PYTHIA',
                   'beauty_comb': '$\it{b}$-jets embedded',
                   'udsg_comb': '$\it{udsg}$-jets embedded',
                  }
        ax.errorbar([],[],label=lab2lab[h.label], **kws)
    pt_caption = f'{jet_pt[0]:.0f} < ' + LABELS['ptjet'] + f'< {jet_pt[1]:.0f}' + LABELS['gevc']
    ax.text(0.98, 0.98, pt_caption, transform=ax.transAxes, va='top', ha='right')
    ax.set_ylabel(r'$\frac{1}{\sigma} \frac{{\rm d}\sigma}{{\rm d} \ln 1/\theta}$', fontsize='x-large')
    ax.legend(frameon=False, fontsize='small', labelspacing=0.1,  
              bbox_to_anchor=(0, 0, 1, 0.94),
              loc='upper left',
             )

    # middle: DC
    ax = axes[1]
    h_dc_py = calc_ratio_1d(h_b_py, h_lf_py, mode=mode)
    h_dc_py.label = 'ratio_py'
    h_dc_comb = calc_ratio_1d(h_b_comb, h_lf_comb, mode=mode)
    h_dc_comb.label = 'ratio_comb'
    ymax = 2
    for h in [h_dc_py, h_dc_comb]:
#         kws = dict(
#                 color=get_prop(colors, h.label), 
#                 ls=get_prop(lses, h.label),
#                 markersize=4, marker='o', lw=0
#         )
        kws = deepcopy(stylings[h.label])
#         kws = kws | dict(
#                 color=get_prop(colors, h.label), 
#                 ls=get_prop(lses, h.label),
#                 markersize=4, marker='o', lw=0
#         )
        kws.update(dict(markersize=6, marker='o', lw=0))

        h.plot1d(ax=ax, yerr=h.variances()**0.5, histtype='errorbar', **kws)
        ax.errorbar([],[],label=h.label.replace('ratio_comb', 'embedded').replace('ratio_py', 'PYTHIA'), **kws)
        ymax = max(ymax, h.values().max())
    ax.legend(frameon=False, fontsize='medium')
    ax.set_ylabel(LABELS['Q_b_udsg'], fontsize='medium')

    # bottom: comb/py
    ax = axes[2]
    h_b_ratio = calc_ratio_1d(h_b_comb, h_b_py, mode=mode)
    h_b_ratio.label = 'ratio_beauty'
    h_lf_ratio = calc_ratio_1d(h_lf_comb, h_lf_py, mode=mode)
    h_lf_ratio.label = 'ratio_udsg'
    for h in [h_b_ratio, h_lf_ratio]:
#         kws = dict(
#                 color=get_prop(colors, h.label), 
#                 ls=get_prop(lses, h.label),
#                 markersize=5, marker='o', lw=0
#         )
        kws = deepcopy(stylings[h.label])
        kws.update(dict(markersize=4, lw=0))
        h.plot1d(ax=ax, yerr=h.variances()**0.5, histtype='errorbar', **kws)
        ax.errorbar([],[],label=h.label.replace('ratio_beauty', '$\it{b}$-jets').replace('ratio_udsg', '$\it{udsg}$-jets'), **kws)
    ax.legend(frameon=False, fontsize='medium', ncol=2, handletextpad=0.1, columnspacing=1)
    ax.set_ylabel(LABELS['Q_comb_py'], fontsize='medium')
    ax.set_xlabel(ax.get_xlabel(), fontsize='large')

    # ax.set_ylim(0 if mode == 'ratio' else -1,ymax*1.1)

#     for ax in axes:
#         ax.grid(ls='--', lw=1, color='lightgrey', axis='y')

    y_ref = 1 if mode == 'ratio' else 0
    for ax in axes[1:]:
        ax.axhline(y_ref, ls='--', c='k', lw=1)
        if mode == 'rel_sym_diff':
            ax.set_ylim(-1.01, 1.3)
    axes[2].set_xlim(binning[1], binning[2])
    # plt.tight_layout()


    ############################  FITTING
    for h_dc,label_py_comb in zip([h_dc_py,h_dc_comb], ['py', 'comb']):
        ax_fit = axes_fits.flatten()[i]

        xs = h_dc.axes.centers[0]
        ys = h_dc.values()
        yerrs = h_dc.variances()**0.5

        mask = (~np.isnan(ys)) & (~np.isnan(yerrs)) & (yerrs > 1e-8)
        xs=xs[mask]
        ys=ys[mask]
        yerrs=yerrs[mask]

        # h_dc_py.plot1d(yerr=h_b_ratio.variances()**0.5, histtype='errorbar',ax=ax)
        kws = deepcopy(stylings['ratio_'+label_py_comb])
        kws['ls'] = ''
        ax_fit.errorbar(xs,ys, yerr=yerrs, **kws)
        ax_fit.axhline(0, color='k', ls='--')

        # initial guess
        a,b,c,d = 1,1,1,1
        a = np.mean(ys[xs < 2])
        d = np.mean(ys[xs > 3])
        a = a if ~np.isnan(a) else 1
        d = d if ~np.isnan(d) else -1
        c = 2.5
        p0 = a,b,c,d
        print(f'initial guess: a={a:.2f}, b={b:.2f}, c={c:.2f}, d={d:.2f}')
        try:
            popt, pcov = curve_fit(sigmoid_gen, xs, ys, sigma=yerrs, p0=p0)
        except:
            print('WARNING: fit failed for ', jet_pt)
            popt,pcov = np.array([0,]*4), np.array([1,]*16).reshape(4,4)
            
        xx = np.linspace(0.8,5,100)

        a,b,c,d = popt
        print(f'fit          : a={a:.2f}, b={b:.2f}, c={c:.2f}, d={d:.2f}')
        kws = deepcopy(stylings['ratio_'+label_py_comb])
        if 'marker' in kws: 
            kws.pop('marker')
        if 'fmt' in kws: 
            kws.pop('fmt')
        thetatr = r'$\theta_{tr}$'
        print('fits ', kws)
        label= 'fit: ' + label_py_comb.replace("comb", "embedded").replace("py", "PYTHIA") + ' ' + thetatr + f' = {c:.2f}'
        ax_fit.plot(xx, sigmoid_gen(xx, *popt), **kws,
#                  label=f'fit: b={b:.2f}, c={c:.2f}',
                    label=label,
                   )
        kws.update(dict(lw=1))
        axes[1].plot(xx, sigmoid_gen(xx, *popt), **kws,
                 label=f'fit: b={b:.2f}, c={c:.2f}'
                )
    #     ax2.plot(xx, func(xx, *popt), 'r-',
    #              label=f'fit: b={b:.2f}, c={c:.2f}'
    #             )
        fit_params[label_py_comb][jet_pt] = popt, pcov
        plt.sca(axes[1])
    axes[0].set_ylim(top=axes[0].get_ylim()[1]*1.6)
#     plt.tight_layout()art
    save_plot(f'dc_pt{jet_pt[0]}-{jet_pt[1]}.png',dirname)
        
    ax_fit.set_ylim(-1.01,1.9);
    ax_fit.legend(frameon=False, fontsize='small', labelspacing=0.1,  
              bbox_to_anchor=(0, 0, 1, 0.9),
                  loc='upper left'
                 )
    ax_fit.text(0.98, 0.98, pt_caption, transform=ax_fit.transAxes, va='top', ha='right')    

for ax_fit in axes_fits[-1,:]:
    ax_fit.set_xlabel(LABELS['lnOneOverTheta'])
for ax_fit in axes_fits[:,0]:
    ax_fit.set_ylabel(r'$Q\,(b,\;udsg)$', fontsize='medium')

plt.sca(ax_fit)
save_plot('fits_all.png',dirname)

In [ ]:
for k1 in erads:
#     if k1 == 'comb': continue
    ls = {'py':'-', 'comb':'--'}[k1]
    for k2 in erads[k1]:
        marker = {'beauty':'o', 'udsg':'x'}[k2]
        for k3 in erads[k1][k2]:
#             if k3 not in ['q25', 'q50', 'q75']: continue
            if k3 not in ['q25', 'q75']: continue
            xx,yy = [],[]
            for k4,vv in erads[k1][k2][k3].items():
                xx.append(np.mean(k4))
                yy.append(vv)
            plt.plot(xx,yy, ls=ls, marker=marker,
                     label='-'.join([k1,k2,k3]))
plt.plot(xx,xx, 'k-', lw=3)
plt.legend()
plt.grid()

save_plot('erads_vs_pt.png',dirname)

In [ ]:
pt_arr = []

mp_py_arr = []
mp_err_py_arr = []

mp_comb_arr = []
mp_err_comb_arr = []

for k,v in fit_params['py'].items():
    pval = v[0]
    pcov = v[1]
    perr = np.sqrt(np.diag(pcov))
    pt_arr.append(np.mean(k))
    mp_py_arr.append(pval[2])
    mp_err_py_arr.append(perr[2])
for k,v in fit_params['comb'].items():
    pval = v[0]
    pcov = v[1]
    perr = np.sqrt(np.diag(pcov))
    mp_comb_arr.append(pval[2])
    mp_err_comb_arr.append(perr[2])


fig,axes = plt.subplots(figsize=(10,4), ncols=2)

ax=axes[0]
kws_py = deepcopy(stylings['ratio_py'])
kws_comb =  deepcopy(stylings['ratio_comb'])
for kws in [kws_py, kws_comb]:
    kws.update({'lw':1, 'ls':''})
ax.errorbar(np.array(pt_arr)-0.5, mp_py_arr, yerr=mp_err_py_arr, **kws_py, label='PYTHIA')
ax.errorbar(np.array(pt_arr)+0.5, mp_comb_arr, yerr=mp_err_comb_arr, **kws_comb, label='embedded')
ax.plot(pt_arr, np.log(np.array(pt_arr)/4.2), '--', color='orange', label=r'$m_{b}$ / '+LABELS['ptjet'])
ax.legend(frameon=False, fontsize='large', ncol=1, handletextpad=0.1, columnspacing=1, loc='upper left')
ax.set_ylim(1,4)
ax.grid(ls=':')
ax.set_xlabel(f"{LABELS['ptjet']} ({LABELS['gevc']})", fontsize='large');
ax.set_ylabel(r'$\theta_{tr}$', fontsize='large')
# save_plot('thetaCrit_vs_pt.png',dirname)



ax = axes[1]
ax.errorbar(mp_py_arr, mp_comb_arr, yerr=mp_err_comb_arr, xerr=mp_err_py_arr, fmt='.', color='k')

xx = np.linspace(1,4,1000)
ax.plot(xx, xx, ':', color='gray')
# plt.legend();
ax.set_xlim(1.0,4)
ax.set_ylim(1.0,4)
plt.grid(ls=':')
ax.set_xlabel(r'$\theta_{tr}^{\rm \; PYTHIA}$', fontsize='large');
ax.set_ylabel(r'$\theta_{tr}^{\rm \; emb}$', fontsize='large');

plt.tight_layout()
save_plot('thetaCrit.png',dirname)

In [ ]:
erad_b = []
erad_err_b = []
erad_udsg = []
erad_err_udsg = []
pt_arr = []

mp_py_arr = []
mp_err_py_arr = []

mp_comb_arr = []
mp_err_comb_arr = []

for pt,v in fit_params['py'].items():
    pval = v[0]
    pcov = v[1]
    perr = np.sqrt(np.diag(pcov))
    pt_arr.append(np.mean(pt)*0.7)
    mp_py_arr.append(pval[2])
    mp_err_py_arr.append(perr[2])
    
    erad_b.append(erads['py']['beauty']['q50'][pt])
    erad_err_b.append((erads['py']['beauty']['q75'][pt]-erads['py']['beauty']['q25'][pt])/2)
    
    erad_udsg.append(erads['py']['udsg']['q50'][pt])
    erad_err_udsg.append((erads['py']['udsg']['q75'][pt]-erads['py']['udsg']['q25'][pt])/2)
    
    
for k,v in fit_params['comb'].items():
    pval = v[0]
    pcov = v[1]
    perr = np.sqrt(np.diag(pcov))
    mp_comb_arr.append(pval[2])
    mp_err_comb_arr.append(perr[2])

    
plt.errorbar(erad_b, mp_py_arr, yerr=mp_err_py_arr, xerr=erad_err_b, fmt='x', label='py-b')
plt.errorbar(erad_udsg, mp_py_arr, yerr=mp_err_py_arr, xerr=erad_err_udsg, fmt='.', label='py-udsg')

plt.errorbar(erad_b, mp_comb_arr, yerr=mp_err_comb_arr, xerr=erad_err_b, fmt='x', label='comb-b')
plt.errorbar(erad_udsg, mp_comb_arr, yerr=mp_err_comb_arr, xerr=erad_err_udsg, fmt='.', label='comb-udsg')
xx = np.linspace(20,200,1000)
plt.plot(xx, np.log(np.array(xx)/4.2), '--')
plt.legend();
plt.ylim(1.0,4)
plt.grid(ls=':')
plt.xlabel('Erad');

save_plot('thetaCrit_vs_erads.png',dirname)